In [1]:
import torch
from torch import nn
import numpy as np
import os
import numpy as np
import itertools
import math, random
random.seed = 42
import numpy as np
import open3d as o3d
import numpy as np
import mpl_toolkits.mplot3d
import matplotlib.pyplot as plt
import pygem
print(pygem.__version__)
from pygem import FFD
from pygem import CustomDeformation
import utils
from utils import *
# 读取 ffd
from path import Path
import random
import os
from bernsetin import *
# from deform import * 
from utils import  *
# import  utils
from path import Path
import scipy.spatial.distance
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import kaleido


2.0.0


# Visualization tools

In [2]:

def read_off(file):
    off_header = file.readline().strip()
    if 'OFF' == off_header:
        n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    else:
        n_verts, n_faces, __ = tuple([int(s) for s in off_header[3:].split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces


def pcshow(xs,ys,zs,colorscale=None):
    clr = np.linspace(0,20,len(xs))
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers',marker=dict(color=clr,colorscale=colorscale))]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2)),
                      selector=dict(mode='markers'))
    fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
    fig.update_layout(scene = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
        )
        )

    fig.show()

def pcwrite(name,xs,ys,zs):
        data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                    mode='markers')]
        fig = visualize_rotate(data)
        fig.update_traces(marker=dict(size=2,
                        line=dict(width=2,
                        color='DarkSlateGrey')),
                        selector=dict(mode='markers'))
        fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
        fig.update_layout(scene = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
        )
        
        )
        fig.write_image(name+'.jpg')

def pc_show_multi(object_coords,control_point_coords):
    """
    xs,ys,zs the coordinates from ML40 object;
    xp,yp,zp  the coordinated from FFD control points 
    """
    xs,ys,zs = object_coords
    xp,yp,zp = control_point_coords


    
    data = [go.Scatter3d(x=xs, y=ys, z=zs,mode='markers',),go.Scatter3d(x=xp, y=yp, z=zp,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers')),

    fig.show()



def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        
        # frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=x_eye, y=y_eye, z=z_eye))))))

    
    fig = go.Figure(data=data,
        layout=go.Layout(
            # updatemenus=[dict(type='buttons',
            #     showactive=False,
            #     y=1,
            #     x=0.8,
            #     # xanchor='left',
            #     # yanchor='bottom',
            #     # pad=dict(t=45, r=10),
            #     # buttons=[dict(label='Play',
            #     #     method='animate',
            #     #     args=[None, dict(frame=dict(duration=50, redraw=True),
            #     #         transition=dict(duration=0),
            #     #         fromcurrent=True,
            #     #         mode='immediate'
            #     #         )]
            #     #     )
            #     # ]
            #     )]
        ),
        frames=frames
    )

    return fig



In [3]:
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):
        # barycentric coordinates on a triangle
        # https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
        
    
    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))
            
        sampled_faces = (random.choices(faces, 
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))
        
        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))
        
        return sampled_points
    

class Normalize(object):
        def __call__(self, pointcloud):
            assert len(pointcloud.shape)==2
            
            norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
            norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

            return  norm_pointcloud
    
    

In [4]:

def handling_inf(A,B):
    with np.errstate(divide='ignore', invalid='ignore'):
        C = A / B
        # 可以参考的对错误处理的方式
        #     arrayC[arrayC == np.inf] = 0  # 对inf的错误进行修正，不会修正-inf
        C[~ np.isfinite(C)] = 0  # 对 -inf, inf, NaN进行修正，置为0

        return C
    


def xyz_to_stu(xyz, origin, stu_axes):
    if stu_axes.shape == (3,):
        stu_axes = np.diag(stu_axes)
        # raise ValueError(
        #     'stu_axes should have shape (3,), got %s' % str(stu_axes.shape))
    # s, t, u = np.diag(stu_axes)
    assert(stu_axes.shape == (3, 3))
    s, t, u = stu_axes

    tu = np.cross(t, u)
    su = np.cross(s, u)
    st = np.cross(s, t)

    diff = xyz - origin

    # TODO: vectorize? np.dot(diff, [tu, su, st]) / ...

    try:
        stu = np.stack([
            np.dot(diff, tu) / np.dot(s, tu),
            np.dot(diff, su) / np.dot(t, su),
            np.dot(diff, st) / np.dot(u, st)
        ], axis=-1)
    except RuntimeWarning:
        print('okk')


    return stu


def stu_to_xyz(stu_points, stu_origin, stu_axes):
    if stu_axes.shape != (3,):
        raise NotImplementedError()
    return stu_origin + stu_points*stu_axes


def get_stu_control_points(dims):
    stu_lattice = utils.mesh3d(
        *(np.linspace(0, 1, d+1) for d in dims), dtype=np.float32)
    stu_points = np.reshape(stu_lattice, (-1, 3))
    return stu_points


def get_control_points(dims, stu_origin, stu_axes):
    stu_points = get_stu_control_points(dims)
    xyz_points = stu_to_xyz(stu_points, stu_origin, stu_axes)
    return xyz_points


def get_stu_deformation_matrix(stu, dims):
    v = utils.mesh3d(
        *(np.arange(0, d+1, dtype=np.int32) for d in dims),
        dtype=np.int32)
    v = np.reshape(v, (-1, 3))

    weights = bernstein_poly(
        n=np.array(dims, dtype=np.int32),
        v=v,
        stu=np.expand_dims(stu, axis=-2))

    b = np.prod(weights, axis=-1)
    return b


def get_deformation_matrix(xyz, dims, stu_origin, stu_axes):
    stu = xyz_to_stu(xyz, stu_origin, stu_axes)
    return get_stu_deformation_matrix(stu, dims)


def get_ffd(xyz, dims, stu_origin=None, stu_axes=None):
    if stu_origin is None or stu_axes is None:
        if not (stu_origin is None and stu_axes is None):
            raise ValueError(
                'Either both or neither of stu_origin/stu_axes must be None')
        stu_origin, stu_axes = get_stu_params(xyz)
    b = get_deformation_matrix(xyz, dims, stu_origin, stu_axes)
    p = get_control_points(dims, stu_origin, stu_axes)
    return b, p, xyz


def deform_mesh(xyz, lattice):
    return trivariate_bernstein(lattice, xyz)


def get_stu_params(xyz):
    minimum, maximum = utils.extent(xyz, axis=0)
    stu_origin = minimum
    # stu_axes = np.diag(maximum - minimum)
    stu_axes = maximum - minimum
    return stu_origin, stu_axes


In [5]:
def _calculate_ffd(vertices, faces, n=3, n_samples=None):
    # import template_FFD.deform as ffd
    # import util3d.mesh.sample as sample
    # stu_origin, stu_axes = ffd.get_stu_params(vertices)

    if n_samples is None:
        points =  PointSampler(3000)((vertices, faces))
        norm_pointcloud = Normalize()(points)
        stu_origin, stu_axes = get_stu_params(norm_pointcloud)

    else:
        pointcloud = PointSampler()((vertices, faces))
    dims = (n,) * 3
    # return ffd.get_ffd(points, dims)
    return get_ffd(norm_pointcloud, dims,stu_origin=stu_origin,stu_axes=stu_axes)

In [6]:
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape) == 2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0)
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return norm_pointcloud

In [7]:

def calculate_ffd(points,n=3):
    # import template_FFD.deform as ffd
    # import util3d.mesh.sample as sample
    # stu_origin, stu_axes = ffd.get_stu_params(vertices)
    dims = (n,) * 3
    # return ffd.get_ffd(points, dims)
    return get_ffd(points, dims)

# Data prepare

In [8]:
from path import Path

path = Path('/Users/wuhongyu/code/Dataset/ModelNet40')

folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};

In [9]:
for cls in folders:
    # get current class folder

    # get all the files in the folder
    file_list = os.listdir(path/"".format(cls))


In [10]:
with open(path/"airplane/train/airplane_0019.off", 'r') as f:

    data  = read_off(f)
    verts, faces  = data 

i,j,k = np.array(faces).T
x,y,z = np.array(verts).T
coords = np.column_stack((x,y,z))


In [11]:
b,p,xyz= _calculate_ffd(np.array(verts),faces,n=3-1)

In [12]:
p.shape

(27, 3)

In [13]:
xyz == np.matmul(b,p)

array([[False, False, False],
       [False, False, False],
       [False, False, False],
       ...,
       [False, False, False],
       [False, False, False],
       [False, False, False]])

In [14]:
dp  =np.random.randint(low=0,high=150,size=(64,3))

In [15]:
# new = np.matmul(b,p)
origin = np.matmul(b,p)


In [16]:
origin.max()

0.9124561531807953

In [17]:
origin.T == xyz.T

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [18]:
pc_show_multi(xyz.T,p.T)

In [25]:

origin = np.matmul(b,p)
pcshow(*(xyz).T)
# pcwrite('test',*(origin).T)

In [39]:
dict_list = []
for i in range(1,9):
    with open(path/"airplane/train/airplane_000{}.off".format(i), 'r') as f:
        verts, faces = read_off(f)
        b,p,xyz= _calculate_ffd(np.array(verts),faces)
        dict_list.append({'b':b,'p':p,'xyz':xyz})


In [40]:
point_cloud_sets = []
for item in dict_list:
    point_cloud_sets.append(item['xyz'])

# Create Model Vanilla

In [42]:
import torch
from torch import nn
from model import Deform_Net,PointNetCls,Contrastive_PointNet
import re

In [43]:
def np_to_tensor(x):
    return torch.from_numpy(x.astype(np.float32))

In [44]:
deform_net1 =  Deform_Net(in_features=128,out_features=(5+1)**3 * 3)
deform_net1.load_state_dict(torch.load('deform_net_1.pth.tar',map_location=torch.device('cpu'))['state_dict'],strict=False)


<All keys matched successfully>

In [45]:
classifier = Contrastive_PointNet()
classifier.load_state_dict(torch.load('best_model.pth.tar',map_location=torch.device('cpu'))['state_dict'],strict=False)


_IncompatibleKeys(missing_keys=['feat.stn.conv1.weight', 'feat.stn.conv1.bias', 'feat.stn.conv2.weight', 'feat.stn.conv2.bias', 'feat.stn.conv3.weight', 'feat.stn.conv3.bias', 'feat.stn.fc1.weight', 'feat.stn.fc1.bias', 'feat.stn.fc2.weight', 'feat.stn.fc2.bias', 'feat.stn.fc3.weight', 'feat.stn.fc3.bias', 'feat.stn.bn1.weight', 'feat.stn.bn1.bias', 'feat.stn.bn1.running_mean', 'feat.stn.bn1.running_var', 'feat.stn.bn2.weight', 'feat.stn.bn2.bias', 'feat.stn.bn2.running_mean', 'feat.stn.bn2.running_var', 'feat.stn.bn3.weight', 'feat.stn.bn3.bias', 'feat.stn.bn3.running_mean', 'feat.stn.bn3.running_var', 'feat.stn.bn4.weight', 'feat.stn.bn4.bias', 'feat.stn.bn4.running_mean', 'feat.stn.bn4.running_var', 'feat.stn.bn5.weight', 'feat.stn.bn5.bias', 'feat.stn.bn5.running_mean', 'feat.stn.bn5.running_var', 'feat.conv1.weight', 'feat.conv1.bias', 'feat.conv2.weight', 'feat.conv2.bias', 'feat.conv3.weight', 'feat.conv3.bias', 'feat.bn1.weight', 'feat.bn1.bias', 'feat.bn1.running_mean', 'feat.

In [71]:
origin_tensor = np_to_tensor(np.array(origin)).unsqueeze(0)
origin_tensor = origin_tensor.transpose(2, 1)

In [72]:
origin_tensor.shape

torch.Size([1, 3, 3000])

In [47]:
point_sets = np_to_tensor(np.array(point_cloud_sets))
point_sets = point_sets.transpose(2, 1)

In [36]:
def deform_point(tensor,classifier,deform_net):
    classifier.eval()
    feats = classifier(tensor)   
    norm_feat = torch.nn.functional.normalize(feats[0], p=2.0, dim = 1)
    dp = deform_net(norm_feat)
    dp = dp.detach().numpy()

    return dp[0]


    



In [73]:
dp = deform_point(origin_tensor,classifier,deform_net1)

In [79]:

origin = np.matmul(b,p)
new =  np.matmul(b,p+dp)
new = Normalize()(new)

In [80]:
pc_show_multi(new.T,(p+dp).T)

In [ ]:
# 造一个新的数据集, 将变形后的物体扩充至 数据集中, 首先应该